# Art Style Classification using CNN

### Imports

In [6]:
import numpy as np
import pandas as pd
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Rescaling, BatchNormalization, GlobalAveragePooling2D, Resizing
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.utils import image_dataset_from_directory
from keras.optimizers import Adam

from keras.applications import MobileNetV2, InceptionV3, DenseNet121, VGG19
from keras.losses import sparse_categorical_crossentropy

### Train/Test Split

In [7]:
# Constants used in the CNN model

data_dir = 'processed'

IMG_HEIGHT, IMG_WIDTH = 512, 512
TARGET_HEIGHT, TARGET_WIDTH = 224, 224

BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2


NUM_CLASSES = 13
EPOCHS = 15

In [8]:
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Art Styles (Class Names): {class_names}")

Found 42385 files belonging to 13 classes.
Using 33908 files for training.
Found 42385 files belonging to 13 classes.
Using 8477 files for validation.
Art Styles (Class Names): ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Western_Medieval']


## Creating CNN model and training it

In [10]:
model = Sequential([
    Resizing(TARGET_HEIGHT, TARGET_WIDTH, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    Rescaling(1./255),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/preprocessing/data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(
    optimizer='adam',
    loss=sparse_categorical_crossentropy,
    metrics=['accuracy']
)
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing_2 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │    44,302,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 13)             │         6,669 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,402,765 (169.38 MB)

 Trainable params: 44,402,765 (169.38 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1230s 1s/step - accuracy: 0.2821 - loss: 2.1171 - val_accuracy: 0.3899 - val_loss: 1.7986
Epoch 2/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1367s 1s/step - accuracy: 0.3943 - loss: 1.8073 - val_accuracy: 0.4828 - val_loss: 1.5340
Epoch 3/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1310s 1s/step - accuracy: 0.4724 - loss: 1.5700 - val_accuracy: 0.5549 - val_loss: 1.3458
Epoch 4/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1397s 1s/step - accuracy: 0.5652 - loss: 1.3031 - val_accuracy: 0.6506 - val_loss: 1.1170
Epoch 5/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1296s 1s/step - accuracy: 0.6689 - loss: 0.9925 - val_accuracy: 0.7359 - val_loss: 0.9175
Epoch 6/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1365s 1s/step - accuracy: 0.7497 - loss: 0.7522 - val_accuracy: 0.7668 - val_loss: 0.8629
Epoch 7/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1455s 1s/step - accuracy: 0.8081 - loss: 0.5791 - val_accuracy: 0.8094 - val_loss: 0.7444
Epoch 8/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1423s 1s/step - accuracy: 0.8508 -

## Evaluating Model on Test Set

In [20]:
loss, accuracy = model.evaluate(val_ds, verbose=0)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Test Loss: 0.7854194641113281
Test Accuracy: 0.8665801286697388


## Saving Model

In [ ]:
import os

SAVED_MODEL_DIR = './models/art_style_cnn.keras'

model.save(SAVED_MODEL_DIR)

print(f"Model saved successfully to directory: {os.path.abspath(SAVED_MODEL_DIR)}")

Model saved successfully to directory: /Users/danish/Documents/GitHub/Art-Styles-Classification/models/art_style_cnn.keras
